# Keras image generators

In [1]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [2]:
num_classes = 10
epochs = 2
batch_size = 16

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)

In [4]:
inputs = keras.Input(shape=(32, 32, 3), name="INPUT")
x = layers.Flatten(name="FLAT")(inputs)
x = layers.Dense(32, activation='relu', name="D1")(x)
x = layers.Dense(16, activation='relu', name="D2")(x)
outputs = layers.Dense(num_classes, name="OUTPUT")(x)
model = keras.Model(inputs=inputs, outputs=outputs, name='model')

In [5]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

In [6]:
# fits the model on batches with real-time data augmentation:
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), 
          steps_per_epoch=len(x_train) / batch_size, epochs=epochs)

  ...
    to  
  ['...']
Train for 3125.0 steps
Epoch 1/2
3125/3125 [==============================] - 63s 20ms/step - loss: 2.1105 - accuracy: 0.2291
Epoch 2/2
3125/3125 [==============================] - 64s 20ms/step - loss: 1.9574 - accuracy: 0.2811


In [ ]:
# here's a more "manual" example
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=32):
        model.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(x_train) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')
model.fit(train_generator,
          steps_per_epoch=2000,
          epochs=50,
          validation_data=validation_generator,
          validation_steps=800)